<a href="https://colab.research.google.com/github/songqsh/ma2071_v01/blob/master/src/lu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sympy import init_printing, Matrix, symbols, eye, printing

lamda = symbols('lamda') # Note that lambda is a reserved word in python, so we use lamda (without the b)

def custom_latex_printer(exp,**options):
    from google.colab.output._publish import javascript
    url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/latest.js?config=default"
    javascript(url=url)
    return printing.latex(exp,**options)

init_printing(use_latex="mathjax",latex_printer=custom_latex_printer)

# LU decomposition of a matrix A

## Definitions

* We will decompose the matrix A into and upper and lower triangular matrix, such that multiplying these will result back into A
$$ A = LU $$
* Given LU decomposition, it is easy to solve for $Ax = b$ by two steps:
  * Solve for $y$ from $Ly = b$ by forward substitution
  * Solve for $x$ from $Ux = y$ by backward substitution
* This approach has $\frac 2 3 n^3$ flops for $Ax = b$, while $x = A^{-1} b$ needs $2 n^3$ flops.

* If $A$ is sparse, the form of $L$ and $U$ are likely to be sparse, while $A^{-1}$ be dense.


## How to obtain LU factorization

* Consider the following matrix of coefficients
$$ \begin{bmatrix} 1 & -2 & 1 \\ 3 & 2 & -2 \\ 6 & -1 & -1 \end{bmatrix} $$
* Successive elementary row operation follow
$$E_{32}*E_{31}*E_{21}*A = U.$$
    * Which is nothing other than matrix multiplication of the __elementary matrices__
    * An elementary matrix is an identity matrix on which one elementary row operation was performed. For instance, to eliminate $A_{ji}$ for some $j>i$, we usually use
    $$E_{ji} = (c \cdot r_i + r_j \to r_j) = I + c e_{ji}.$$
    * row exchange shall not be used.
* Lower triangular matrix is
$$L = (E_{32}*E_{31}*E_{21})^{-1}.$$
  * Note that $(E_{ji})^{-1} = I - c e_{ji}$ for any $j>i$.

In [2]:
A = Matrix([[1, -2, 1], [3, 2, -2], [6, -1, -1]])
A

⎡1  -2  1 ⎤
⎢         ⎥
⎢3  2   -2⎥
⎢         ⎥
⎣6  -1  -1⎦

In [3]:
'''=============
E21 = (-3 * r1 + r2 --> r2) = -3 * e21 + I
==============='''

E21 = eye(3); E21[1,0] = -3
E21

⎡1   0  0⎤
⎢        ⎥
⎢-3  1  0⎥
⎢        ⎥
⎣0   0  1⎦

In [4]:
E21*A

⎡1  -2  1 ⎤
⎢         ⎥
⎢0  8   -5⎥
⎢         ⎥
⎣6  -1  -1⎦

In [5]:
'''=============
E31 = (-6 * r1 + r3 --> r3) = -6 * e31 + I
==============='''

E31 = eye(3); E31[2,0] = -6
E31

⎡1   0  0⎤
⎢        ⎥
⎢0   1  0⎥
⎢        ⎥
⎣-6  0  1⎦

In [6]:
E31*E21*A

⎡1  -2  1 ⎤
⎢         ⎥
⎢0  8   -5⎥
⎢         ⎥
⎣0  11  -7⎦

In [7]:
'''=============
E32 = (-11/8 * r2 + r3 --> r3) = -11/8 * e32 + I
==============='''

E32 = eye(3); E32[2,1] = -11/8
E32

⎡1    0     0⎤
⎢            ⎥
⎢0    1     0⎥
⎢            ⎥
⎣0  -1.375  1⎦

In [8]:
'''=============
it ends up with upper triangular U
==============='''
E32*E31*E21*A

⎡1  -2    1   ⎤
⎢             ⎥
⎢0  8     -5  ⎥
⎢             ⎥
⎣0  0   -0.125⎦

In [9]:
'''===========
lower triangular L
==============='''
E21.inv()*E31.inv()*E32.inv()

⎡1    0    0⎤
⎢           ⎥
⎢3   1.0   0⎥
⎢           ⎥
⎣6  1.375  1⎦

There is one-shot method for LU factorization

In [10]:
L, U, X = A.LUdecomposition()
L

⎡1   0    0⎤
⎢          ⎥
⎢3   1    0⎥
⎢          ⎥
⎣6  11/8  1⎦

In [11]:
U

⎡1  -2   1  ⎤
⎢           ⎥
⎢0  8    -5 ⎥
⎢           ⎥
⎣0  0   -1/8⎦